In [ ]:
#################################
#             주의사항            #
#################################

# 1. 작업이 끝나면 반드시 연결을 닫아주세요. (conn.close())
# 2. 한글 입력이 가능하도록 설정을 변경했습니다. (RDS - 파라미터그룹 - utf8mb4)
# 3. 새로 생성되는 테이블의 CHAR SET 은 UTF-8로 설정해주세요.

In [53]:
import pymysql
import pandas as pd

In [39]:
# 연결 RDS 정보

host = "team06-antifragile-db.cxuncqkdvk3h.us-east-1.rds.amazonaws.com"
port = 3306
username = "admin"
database = "antifragile"
password = "antifragile1234"

In [87]:
# pymysql을 통한 로컬 - AWS RDS 연결

conn = pymysql.connect(host = host, user = username, port = port, database = database, password = password)
cursor = conn.cursor()

In [73]:
# 명령문 실행

## 명령문 작성
sql = '''
show tables;
'''

## 명령문 실행
cursor.execute(sql)

## 실행 결과 출력
response = cursor.fetchall()
for data in response:
    print(data)

('cam',)
('images',)
('log',)
('object',)
('user',)


In [89]:
# sql 연결 해제

conn.close()